# DIT Natural Language Processing lesson 2025

## Using character-level representations for classification




In [ ]:
# Importing the dependencies
import glob
import numpy as np
import os
import tarfile

from keras.models import Sequential
from keras.layers import Dense, Dropout, Input, Flatten, LSTM

from random import shuffle
from tqdm.auto import tqdm
from urllib import request

In [ ]:
# CONSTANTS

PATH_TO_CORPUS = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
CORPUS_FILE_NAME = "aclImdb_v1.tar.gz"

CORPUS_PATH = "aclImdb/train"

VALID = set([x for x in 'abcdefghijklmnopqrstuvwxyz0123456789"\'?!.,:; '])
VALID

In [ ]:
# Download and set up the corpus

def download_file(url_to_file, path_to_file):
  if os.path.isfile(path_to_file):
    print("A local copy of the file exists already:", path_to_file, "\nDoing nothing")
  else:
    request.urlretrieve(url_to_file, path_to_file)

# Downloading and untaring the corpus

download_file(PATH_TO_CORPUS, CORPUS_FILE_NAME)
with tarfile.open(CORPUS_FILE_NAME) as f:
  f.extractall(path=".")

In [ ]:
# Loading the data

def pre_process_data(filepath):
    """
    Load pos and neg examples from separate dirs then shuffle them
    together.
    """
    positive_path = os.path.join(filepath, 'pos')
    negative_path = os.path.join(filepath, 'neg')
    pos_label = 1
    neg_label = 0
    dataset = []
    for filename in glob.glob(os.path.join(positive_path, '*.txt')):
      with open(filename, 'r') as f:
            dataset.append((pos_label, f.read()))
    for filename in glob.glob(os.path.join(negative_path, '*.txt')):
        with open(filename, 'r') as f:
            dataset.append((neg_label, f.read()))
    shuffle(dataset)
    return dataset

def collect_expected(dataset):
    """Extracting the expected output for all the instances"""
    return [sample[0] for sample in dataset]

In [ ]:
# Loading instances and expected classes (as usual)
dataset = pre_process_data(CORPUS_PATH)
expected = collect_expected(dataset)

In [ ]:
def avg_len(data):
    """Computes the average length of the data"""
    total_len = 0
    for sample in data:
        total_len += len(sample[1])
    return total_len/len(data)
avg_len(dataset)

The average word length is 202.44 (**Homework:** don't believe me and go find yourself).

That is, we would unroll the network **6.5x**!

In [ ]:
def clean_data(data):
    """Casefold, replace unknowns with UNK, and listify """
    new_data = []
    # VALID = 'abcdefghijklmnopqrstuvwxyz0123456789"\'?!.,:; '
    for sample in data:
        new_sample = []
        for char in sample[1].lower():  # Just grab the string, not the label
            # Not extremely efficient procedure
            if char in VALID:
                new_sample.append(char)
            else:
                new_sample.append('UNK')

        new_data.append(new_sample)
    return new_data

**Homework**: turn the process to determine if a character is VALID more efficient

(I did already, by switching VALID into a set)

In [ ]:
# We are pading and truncating outside of the Collator this
# time to set the data up in advance. As a result, the
# Collator wont pad/truncate. It will simply feed the data
# to fit by batches.

def char_pad_trunc(data, maxlen):
    """ We truncate to maxlen or add PAD tokens """
    new_dataset = []
    for sample in data:
        if len(sample) > maxlen:
            new_data = sample[:maxlen]
        elif len(sample) < maxlen:
            pads = maxlen - len(sample)
            new_data = sample + ['PAD'] * pads
        else:
            new_data = sample
        new_dataset.append(new_data)
    return new_dataset

# Padding and truncating the input is not strictly necessary for RNNs; we
# due to the recursion strategy we are applying here and to have a fair
# comparison against the CNN (same input length)
class Collator:
  def __init__(self,
                # maxlen,
                batch_size,
                ) -> None:
    # self.maxlen = maxlen
    self.batch_size = batch_size

  # Since this time I am padding/truncating beforehand,
  # I do not need to do it again
  # def padding_and_truncating(self, x, y):
  #   """
  #   Add zeros at the end of the representation for short instances,
  #   truncate longer ones to the maxlen
  #   """
  #   # vec_dim = len(x[0][0])
  #   # N = len(x)
  #   # X = np.zeros((N, self.maxlen, vec_dim))  # preallocate a np array
  #   # Y = np.array(y)

  #   # for i, tokens in enumerate(x):
  #   #   length = min(len(tokens), self.maxlen)
  #   #   if length > 0:
  #   #     X[i, :length] = np.asarray(tokens[:length])  # fill the np array
  #   # return X, Y
  #   return x, y

  def collate(self, X, Y, N, epochs = 1):
    """
    This method is used to feed batches into the `model.fit` method
    """
    for _ in range(epochs):
      """
      This `for _ in range(epochs):` loop is here because
      the `for` below needs to be able to be called `epochs` times
      so each time the `for` is called the iterator is replenished
      for the new epoch
      """
      for start in range(0, N, self.batch_size):
        end = start + self.batch_size
        x_batch = X[start:end]
        y_batch = Y[start:end]
        # x_batch, y_batch = self.padding_and_truncating(
        #   X[start:end],
        #   Y[start:end]
        #   )
        yield x_batch, y_batch

In [ ]:
maxlen = 800
# maxlen = 1500
batch_size = 32

collator = Collator(
    # maxlen = maxlen,
    batch_size = batch_size)

In [ ]:
# Producing the one-hot encodings (no embeddings here)
def create_dicts(data):
    """ Modified from Keras LSTM example"""
    chars = set()
    for sample in data:
        chars.update(set(sample))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))
    return char_indices, indices_char

In [ ]:
def onehot_encode(dataset, char_indices, maxlen):
    """
    One hot encode the tokens

    Args:
        dataset  list of lists of tokens
        char_indices  dictionary of {key=character, value=index to use encoding vector}
        maxlen  int  length of each sample
    Return:
        np array of shape (samples, tokens, encoding length)
    """
    # print(len(dataset), maxlen)
    X = np.zeros((len(dataset), maxlen, len(char_indices.keys())))
    for i, sentence in enumerate(dataset):
        # print(i)
        for t, char in enumerate(sentence):
            X[i, t, char_indices[char]] = 1
    return X

In [ ]:
# Load and preprocess the data

listified_data = clean_data(dataset)


common_length_data = char_pad_trunc(listified_data, maxlen)
char_indices, indices_char = create_dicts(common_length_data)
encoded_data = onehot_encode(common_length_data, char_indices, maxlen)

# char_indices, indices_char = create_dicts(listified_data)
# encoded_data = onehot_encode(listified_data, char_indices, maxlen)
print(char_indices)
print(indices_char)

In [ ]:
# Split the data
split_point = int(len(encoded_data)*.8)

x_train = encoded_data[:split_point]
y_train = np.array(expected[:split_point])
x_test = encoded_data[split_point:]
y_test = np.array(expected[split_point:])
# Pay attention: in the book they forgot to turn y_[train|test] into numpy arrays

In [ ]:
# A quick view to the first instance
x_train[0]

In [ ]:
# Shape of the resulting array
x_train.shape

In [ ]:
# How many instances do we have?
len(x_train)

**Q: What is  the size of the vocabulary?**

In [ ]:
# Building the network

# Embedding,
num_neurons = 40

print('Build model...')
model = Sequential()

model.add(Input([maxlen, len(char_indices.keys())]))

model.add(LSTM(
    num_neurons,
    return_sequences=True
    # ,
    # input_shape=(maxlen, len(char_indices.keys()))
    )
    )

model.add(Dropout(.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy'])
model.summary()

[rmsprop](https://keras.io/api/optimizers/rmsprop/)

In [ ]:
# Training the network
# batch_size = 32
epochs = 10

# Original fit (withput Collator-based batching)
# model.fit(x_train, y_train,
#     batch_size=batch_size,
#     epochs=epochs,
#     validation_data=(x_test, y_test)
#     )

model.fit(
    collator.collate(x_train, y_train, N = len(x_train), epochs=epochs),
    steps_per_epoch=len(x_train) // batch_size,
    validation_data=collator.collate(x_test, y_test, N = len(x_test), epochs=epochs),
    validation_steps=len(x_test) // batch_size,
    epochs=epochs,
    )
# This would take between 5 and 10 minutes per epoch, depending on the hardware!

In [ ]:
# Saving the model
model_structure = model.to_json()
with open("char_lstm_model3.json", "w") as json_file:
    json_file.write(model_structure)
model.save_weights("char_lstm.weights.h5")

Back to the slides